# RequestGeneratorV3

In [18]:
# import random, json, math, os
# import numpy as np
# from scheduler_utils import overlap_time_segments, trim_time_segments
# import math

# SECONDS_IN_DAY = 24 * 60 * 60

# # Scheduling Horizon in seconds
# # 7 days - Taken from provided input file
# default_horizon_days = 7

# # Time Discretization Slice Size in seconds
# # 5 minutes - Taken from provided input file
# default_slice_size = 300

# # Approximated from times found in telescope_times.json
# # Stored as floats representing the rounded hour in 24hr time
# # E.g. "04:00:00" = 4.0, "23:30:00" = 23.5
# estimated_telescope_times = {
#     "COJ": {
#         "start": 8,
#         "end": 20
#     },
#     "OGG": {
#         "start": 4,
#         "end": 17.5
#     },
#     "LSC": {
#         "start": 23,
#         "end": 11
#     },
#     "CPT": {
#         "start": 17,
#         "end": 4.5
#     },
#     "TFN": {
#         "start": 18.5,
#         "end": 8
#     },
#     "TLV": {
#         "start": 15.5,
#         "end": 3.5
#     }
# }

# telescopes_2m = ["COJ", "OGG"]


# class Injection(object):
#     def __init__(self, injection_time, injection_type, data=None):
#         self.injection_time = injection_time
#         self.injection_type = injection_type
#         self.data = data

#     def __str__(self):
#         return f"{self.injection_type}_{self.injection_time}"
    
#     def __lt__(self, other):
#         if self.injection_time == other.injection_time:
#             return self.type_lt(other)
#         else:
#             return self.injection_time < other.injection_time

#     def __gt__(self, other):
#         if self.injection_time == other.injection_time:
#             return self.type_gt(other)
#         else:
#             return self.injection_time > other.injection_time

#     def __eq__(self, other):
#         if self.injection_time == other.injection_time:
#             return self.type_eq(other)
#         else:
#             return False

#     def type_lt(self, other):
#         if self.injection_type == "request": # Either Request vs Request, or Request vs Resource
#             return False
#         elif other.injection_type == "resource": # Either Resource vs Resource, or Request vs Resource
#             return False
#         else:
#             return True # Must be Resource vs Request

#     def type_gt(self, other):
#         if self.injection_type == "resource":
#             return False
#         elif other.injection_type == "request":
#             return False
#         else: return True

#     def type_eq(self, other):
#         return self.injection_type == other.injection_type


#     def to_json(self):
#         return {
#             "injection_time": self.injection_time,
#             "injection_type": self.injection_type,
#             "injection_data": self.data
#         }


# class ResourceInjection(Injection):
#     def __init__(self, injection_time, resource_data):
#         super().__init__(injection_time, "resource", resource_data)


# class RequestInjection(Injection):
#     def __init__(self, injection_time, request_data):
#         super().__init__(injection_time, "request", request_data)


# class RequestGeneratorV3(object):
#     def __init__(self, now=0, horizon_days=default_horizon_days,
#                  num_telescopes=1, num_networks=1, use_2m_telescopes=False,
#                  num_proposals=5, proposals_dict=None,
#                  slice_size=default_slice_size):

#         # Store necessary parameters
#         self.now = now # The start time of the scheduler, in seconds
#         # Must be from a 24hr clock (so < 1 day, so < 24*60*60)
#         # Defaults to be midnight.

#         self.horizon_days = horizon_days
#         self.horizon = self.now + horizon_days*24*60*60
#         self.slice_size=slice_size

#         # Generating Telescope Opening Hours
#         if use_2m_telescopes:
#             self.resources = generate_2m_telescopes()
#             self.networks = self.generate_sub_networks(num_networks=1)
#         else:
#             self.resources = self.generate_telescope_times(num_telescopes=num_telescopes)
#             self.networks = self.generate_sub_networks(num_networks=num_networks)

#         # Generating Proposal Information
#         if proposals_dict != None:
#             self.proposals = proposals_dict
#         else:
#             self.proposals = self.generate_default_proposals(num_proposals)
#             # self.generate_default_proposals(num_proposals)

#         # Make variables for used data
#         self.injections = []
#         self.request_count = 0

#         # Load request pattern information
#         pattern_data = json.load(open("../archive_scraper/observation_patterns_v1.json", "r"))
#         self.observation_patterns = sorted(pattern_data.values(), key=lambda x: x["cumulative_probability"])


#     def generate_telescope_times(self, num_telescopes):
#         telescopes = {}
#         telescope_options = list(estimated_telescope_times.keys())

#         for i in range(num_telescopes):
#             # Assume the opening hours for a random telescope site
#             site = random.choice(telescope_options)
#             start_time = int(estimated_telescope_times[site]["start"] * 60 * 60)
#             end_time = int(estimated_telescope_times[site]["end"] * 60 * 60)

#             telescopes[f"telescope_{i}"] = self.propagate_telescope_times(start_time, end_time)

#         return telescopes


#     def propagate_telescope_times(self, start_time, end_time):
#         if end_time < start_time:
#             start_time -= SECONDS_IN_DAY

#         open_hours = [{"start": start_time + i*SECONDS_IN_DAY, "end": end_time + i*SECONDS_IN_DAY} \
#             for i in range(self.horizon_days+2)]

#         open_hours = trim_time_segments(open_hours, self.now, self.horizon)

#         return open_hours


#     def generate_2m_telescopes(self):
#         telescopes = {}
#         for site in ["COJ", "OGG"]:
#             start_time = estimated_telescope_times[site]["start"] * 60 * 60
#             end_time = estimated_telescope_times[site]["end"] * 60 * 60

#             telescopes[site] = self.propagate_telescope_times(start_time, end_time)

#         return {"network_0": telescopes}


#     def generate_sub_networks(self, num_networks):
#         # A network is a collection of telescopes that can share observation requests,
#         # but do not share observation requests between networks
#         # e.g. the 2m telescopes vs the 1m telescopes vs the 0.4m telescopes.
#         return np.array_split(list(self.resources.keys()), num_networks)

#         # networks = {}
#         # for i in range(len(num_networks)):
#         #     network_telescopes = self.generate_telescope_times(num_telescopes=num_telescopes_per_network)
#         #     networks[f"network_{i}"] = network_telescopes
#         # return networks


#     def generate_default_proposals(self, num_proposals):
#         proposals = {}
#         for i in range(num_proposals):
#             proposal_name = f"proposal_{i}"
#             tac_priority = random.randint(4, 6) * 5
#             proposals[proposal_name] = {"tac_priority": tac_priority}
#         return proposals


#     # def generate_requests(self, num_requests, injection_time, 
#     #                       request_min_length=60, request_max_length=10800,
#     #                       telescope_network=None):
#     #     requests = {}
#     #     for i in range(num_requests):
#     #         if telescope_network == None:
#     #             windows = {r: time_segments(injection_time, self.end_time, 
#     #                                         1, 8) for r in self.resources}
#     #         else:
#     #             windows = {r: time_segments(injection_time, 
#     #                         self.end_time, 1, 8) for r in self.networks[telescope_network]}

#     #         duration = random.randint(request_min_length, request_max_length)
#     #         proposal = random.choice(list(self.proposals.keys()))

#     #         requests[self.request_count] = {
#     #             "windows": windows,
#     #             "duration": duration,
#     #             "proposal": proposal,
#     #             "resID": self.request_count
#     #         }

#     #         self.request_count += 1     # Make sure that each request has a unique ID

#     #     return requests


#     def get_pattern(self):
#         rnum = random.random()
#         for p in self.observation_patterns:
#             if rnum <= p["cumulative_probability"]:
#                 return p["pattern"]


#     def generate_request(self, injection_time, telescope_network=None):
#         # Pick a pattern
#         # Pick a time a certain distance from the injection time
#         # Pick an availability window size
#         # Pick a network of telescopes
#         pattern = self.get_pattern()
#         total_pattern_length = int(sum(pattern))
        
#         # TO FIX: Work out a way of systematically generating an availability window.
#         availability_window_length = int(round(((random.random() * 9) + 1) * total_pattern_length))

#         start_time = int(round(random.random() * self.horizon * 0.5))

#         if telescope_network == None:
#             windows = {r: [{"start": start_time, "end": start_time + availability_window_length}] \
#                 for r in self.resources}
#         else:
#             windows = {r: [{"start": start_time, "end": start_time + availability_window_length}] \
#                 for r in self.networks[telescope_network]}

#         proposal = random.choice(list(self.proposals.keys()))

#         requestID = self.request_count

#         self.request_count += 1
        
#         return {
#             "windows": windows,
#             "duration": total_pattern_length,
#             "proposal": proposal,
#             "resID": requestID
#         }
        

#     # def generate_single_request_injection(self, inj_time, num_requests, 
#     #                                     min_length=60, max_length=10800, network=0):
#     #     requests = self.generate_requests(num_requests, inj_time, 
#     #                                       min_length, max_length, network)
#     #     self.injections.append(RequestInjection(inj_time, requests))

#     def generate_single_request_injection(self, inj_time, num_requests, network=0):
#         new_requests = [self.generate_request(inj_time, network) for i in range(num_requests)]
#         requests = {r["resID"]: r for r in new_requests}
#         self.injections.append(RequestInjection(inj_time, requests))


#     def generate_requests_injections(self, injection_dict):
#         for injection_time, num_requests in injection_dict.items():
#             self.generate_single_request_injection(injection_time, num_requests)


#     def generate_single_telescope_closure(self, inj_time, resource, 
#                                           end_time=None, max_close_time=None):
#         closure_start = inj_time
#         if end_time != None:
#             closure_end = end_time

#         else:
#             length = random.randint(0, self.horizon - inj_time)
#             if max_close_time != None:
#                 length = min(length, max_close_time)
#             closure_time = closure_start + length

#         data = {
#             "start_time": closure_start,
#             "end_time": closure_end,
#             "resource": resource
#         }

#         self.injections.append(ResourceInjection(closure_start, data))


#     def generate_telescope_closures(self, num_closures, max_close_time):
#         for i in range(num_closures):
#             inj_time = random.randint(self.start_time, self.horizon-2000)
#             self.generate_single_telescope_closure(inj_time, "t1", 
#                                                    max_close_time=max_close_time)


#     def output_to_json(self):
#         output = {
#             "input_parameters": {
#                 "start_time": self.now,
#                 "slice_size": self.slice_size,
#                 "horizon": self.horizon,
#                 "resources": self.resources,
#                 "proposals": self.proposals
#             },
#             "injections": [inj.to_json() for inj in self.injections]
#         }

#         return output


#     def save_to_file(self, filename=None, dirname="sample_input"):
#         if filename == None:
#             file_prefix = "sample_input_v3_"
#             file_index = 0
#             while True:
#                 filename = f"{file_prefix}{file_index}.json"
#                 temp_path = f"{dirname}/{filename}.json"
#                 if os.path.isfile(temp_path):
#                     file_index += 1
#                 else:
#                     break

#         output = self.output_to_json()

#         filepath = os.path.join(dirname, filename)

#         with open(filepath, "w") as f:
#             json.dump(output, f, indent=4)
#         print(f"Saved to file at '{filepath}'")


#     def auto_run(self, request_injection_dict, num_telescope_closures):
#         self.generate_input_params()
#         self.generate_requests_injections(request_injection_dict)
#         max_closure_length = int(self.horizon / 10)
#         self.generate_telescope_closures(num_telescope_closures, 
#                                          max_closure_length)
#         self.injections.sort()
#         print("Finished Request Generation V2")
#         self.save_to_file()

In [1]:
from generate_requests_v3 import RequestGeneratorV3

# Generate Input Files

## General Parameters

In [2]:
num_repeats = 10

## Scaling Number of Requests

In [3]:
# for num_requests in [100, 500, 1000, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000]:
# for num_requests in [100,200, 300, 400, 500]:
for num_requests in [100]:
    for i in range(3):
        rg = RequestGeneratorV3(i)
        rg.generate_single_request_injection(0, num_requests)
        filename = "performanceTest1_{}Requests_{}.json".format(num_requests, i)
        rg.save_to_file(filename, "test_input")

Saved to file at 'test_input\performanceTest1_100Requests_0.json'
Saved to file at 'test_input\performanceTest1_100Requests_1.json'
Saved to file at 'test_input\performanceTest1_100Requests_2.json'


### Testing

In [22]:
for num_requests in [100, 500, 1000, 5000, 10000, 50000, 100000]:
    for i in range(3):
        random.seed(i)
        rg = RequestGeneratorV3()
        rg.generate_single_request_injection(0, num_requests)
        filename = "performanceTest1_{}Requests_{}.json".format(num_requests, i)
        rg.save_to_file(filename, "benchmark_input")

Saved to file at 'benchmark_input\performanceTest1_100Requests_0.json'
Saved to file at 'benchmark_input\performanceTest1_100Requests_1.json'
Saved to file at 'benchmark_input\performanceTest1_100Requests_2.json'
Saved to file at 'benchmark_input\performanceTest1_500Requests_0.json'
Saved to file at 'benchmark_input\performanceTest1_500Requests_1.json'
Saved to file at 'benchmark_input\performanceTest1_500Requests_2.json'
Saved to file at 'benchmark_input\performanceTest1_1000Requests_0.json'
Saved to file at 'benchmark_input\performanceTest1_1000Requests_1.json'
Saved to file at 'benchmark_input\performanceTest1_1000Requests_2.json'
Saved to file at 'benchmark_input\performanceTest1_5000Requests_0.json'
Saved to file at 'benchmark_input\performanceTest1_5000Requests_1.json'
Saved to file at 'benchmark_input\performanceTest1_5000Requests_2.json'
Saved to file at 'benchmark_input\performanceTest1_10000Requests_0.json'
Saved to file at 'benchmark_input\performanceTest1_10000Requests_1.js

## Scaling Number of Telescopes

In [53]:
for num_telescopes in [1, 2, 3, 5, 10, 15, 20]:
    # for num_requests in [500, 1000, 10000]:
# for num_telescopes in [1, 3, 5]:
    for num_requests in [100]:
        for i in range(num_repeats):
            random.seed(i)
            rg = RequestGeneratorV3(num_telescopes=num_telescopes)
            rg.generate_single_request_injection(0, num_requests)
            filename = "performanceTest2_{}Telescopes_{}Requests_{}.json".format(num_telescopes, num_requests, i)
            rg.save_to_file(filename, "performance_input")

Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_0.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_1.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_2.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_3.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_4.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_5.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_6.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_7.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_8.json'
Saved to file at 'performance_input\performanceTest2_1Telescopes_100Requests_9.json'
Saved to file at 'performance_input\performanceTest2_2Telescopes_100Requests_0.json'
Saved to file at 'performance_input\performanceTest2_2Telescopes_

## Scaling Slice Size

In [54]:
for slice_size in [600, 300, 240, 180, 120, 60, 30]:
# for slice_size in [600, 300, 180]:
    for num_requests in [100]:
        for i in range(num_repeats):
            random.seed(i)
            rg = RequestGeneratorV3(slice_size=slice_size)
            rg.generate_single_request_injection(0, num_requests)
            filename = "performanceTest3_{}SliceSize_{}Requests_{}.json".format(slice_size, num_requests, i)
            rg.save_to_file(filename, "performance_input")

Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_0.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_1.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_2.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_3.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_4.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_5.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_6.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_7.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_8.json'
Saved to file at 'performance_input\performanceTest3_600SliceSize_100Requests_9.json'
Saved to file at 'performance_input\performanceTest3_300SliceSize_100Requests_0.json'
Saved to file at 'performance_input\performanceTest3_3

## Scaling Number of Networks

In [56]:
num_telescopes = 8
# num_telescopes = 4
# num_requests = 1000
num_requests = 25
for num_networks in [1, 2, 4, 8]:
# for num_networks in [1, 2, 4]:
    for i in range(num_repeats):
        random.seed(i)
        rg = RequestGeneratorV3(num_telescopes=num_telescopes, num_networks=num_networks)
        for network in range(num_networks):
            rg.generate_single_request_injection(0, num_requests, network=network)
        filename = "performanceTest5_{}Networks_{}Telescopes_{}RequestsPerNetwork_{}.json".format(num_networks, num_telescopes, num_requests, i)
        rg.save_to_file(filename, "performance_input")

Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_0.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_1.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_2.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_3.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_4.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_5.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_6.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_7.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8Telescopes_25RequestsPerNetwork_8.json'
Saved to file at 'performance_input\performanceTest5_1Networks_8

## Scaling Number of Networks (with scaling request number)

In [57]:
num_telescopes = 8
# num_telescopes = 4
# gross_requests = 8000
gross_requests = 200
for num_networks in [1, 2, 4, 8]:
# for num_networks in [1, 2, 4]:
    requests_per_network = int(gross_requests / num_networks)
    for i in range(num_repeats):
        random.seed(i)
        rg = RequestGeneratorV3(num_telescopes=num_telescopes, num_networks=num_networks)
        for network in range(num_networks):
            rg.generate_single_request_injection(0, requests_per_network, network=network)
        filename = "performanceTest6_{}Networks_{}Telescopes_{}GrossRequests_{}.json".format(num_networks, num_telescopes, gross_requests, i)
        rg.save_to_file(filename, "performance_input")

Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_0.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_1.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_2.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_3.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_4.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_5.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_6.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_7.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_8.json'
Saved to file at 'performance_input\performanceTest6_1Networks_8Telescopes_200GrossRequests_9.json'


## Scaling Scheduling Horizon

In [21]:
for horizon_days in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] :
    for num_requests in [1000, 2000, 5000, 10000]:
        for i in range(num_repeats):
            random.seed(i)
            rg = RequestGeneratorV3(horizon_days=horizon_days)
            rg.generate_single_request_injection(0, num_requests)
            filename = "performanceTest7_{}HorizonDays_{}Requests_{}.json".format(horizon_days, num_requests, i)
            rg.save_to_file(filename, "performance_input")

Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_0.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_1.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_2.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_3.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_4.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_5.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_6.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_7.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_8.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_1000Requests_9.json'
Saved to file at 'performance_input\performanceTest7_1HorizonDays_2000Requests_0.json'
Saved to file at 'performance_input\perform

# Variable Window Test

Conditions for the Variable Window Test (and for the Priority Banding Test):
* Preload majority of the requests at the beginning, in one injection
* Load a small amount of requests in on a regular basis, to simulate a dynamic and overloaded scheduler
* Make sure that the requests are extending past the Simulator Horizon (which potentially needs to be distinct from the Scheduler Horizon)
  * Do we need to simulate an entire observing semester? If so, is there a way of simplifying this? Are we satisfied with just a week, or two weeks?
* Add in random closures of the telescopes, based on weather. See if there is some historical data for how much the telescopes are actually open.

In [15]:
num_scenarios = 3
total_num_requests = 100
bulk_fraction = 0.7
num_injections = 10

def split_requests(num_requests, num_splits):
    # print(num_requests, num_splits)
    split_size = int(num_requests / num_splits)
    remainder = num_requests % num_splits
    print(num_requests, num_splits, split_size, remainder)
    splits = []
    for i in range(num_splits):
        if i < remainder:
            splits.append(split_size + 1)
        else:
            splits.append(split_size)
    return splits

bulk_request_num = round(total_num_requests * bulk_fraction)
print("Bulk Requests:", bulk_request_num)
remaining_request_num = round(total_num_requests * (1 - bulk_fraction))
print("Remaining Requests:", remaining_request_num)
injections = split_requests(remaining_request_num, num_injections)
print("Injections:", injections)
print(sum(injections))

Bulk Requests: 70
Remaining Requests: 30
30 10 3 0
Injections: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
30


In [16]:
for i in range(num_scenarios):
    random.seed(i)
    rg = RequestGeneratorV3(num_telescopes=2)
    # Generate upfront bulk request injection
    rg.generate_single_request_injection(0, bulk_request_num)
    injection_time = 0
    for request_num in split_requests(remaining_request_num, num_injections):
        injection_time += 60*60*3 # Space injections by 3 hours - WHY??? No reason currently.
        rg.generate_single_request_injection(injection_time, request_num)
    filename = "variableWindowsTest_{}.json".format(i)
    rg.save_to_file(filename, "variableWindows_input")

30 10 3 0
Saved to file at 'variableWindows_input\variableWindowsTest_0.json'
30 10 3 0
Saved to file at 'variableWindows_input\variableWindowsTest_1.json'
30 10 3 0
Saved to file at 'variableWindows_input\variableWindowsTest_2.json'


# Testing

In [4]:
obs_data = json.load(open("../archive_scraper/observation_patterns_v2.json", "r"))

In [5]:
sorted_obs_data = sorted(obs_data.values(), key=lambda x: x["cumulative_probability"])

In [10]:
rnum = random.random()

for p in sorted_obs_data:
    if rnum < p["cumulative_probability"]:
        print(p["pattern"])
        print(sum(p["pattern"]))
        break

[1800.0, 60.0, 20.0]
1880.0


In [7]:
sorted_obs_data[0]

{'pattern': [200.0],
 'count': 197,
 'cumulative_probability': 0.08011386742578284}

In [16]:
a = []
for p in sorted_obs_data:
    # print(sum(p["pattern"]))
    a.append(sum(p["pattern"]))

In [13]:
max(a)

10080.0

In [14]:
min(a)

0.0

In [22]:
np.percentile(a, 25)

440.0

In [15]:
24*60*60

86400

In [53]:
def get_pattern():
    rnum = random.random()
    for p in sorted_obs_data:
        if rnum < p["cumulative_probability"]:
        # print(p["pattern"])
        # print(sum(p["pattern"]))
            return sum(p["pattern"])

In [54]:
def generate_req():
    pattern = get_pattern()
    awlength = round(((random.random() * 10) + 1) * pattern)
    return (pattern, awlength)

In [56]:
for i in range(100):
    print(generate_req())

(190.0, 863)
(240.0, 2563)
(3400.0, 36338)
(270.0, 1316)
(2400.0, 24252)
(684.0, 4380)
(480.0, 1531)
(190.0, 1365)
(6490.0, 45771)
(490.0, 3985)
(200.0, 1109)
(1800.0, 3481)
(3700.0, 20982)
(80.0, 813)
(300.0, 430)
(60.0, 522)
(300.0, 664)
(4900.0, 11989)
(480.0, 4531)
(4900.0, 5895)
(420.0, 2423)
(4910.0, 23411)
(1900.0, 13391)
(360.0, 3407)
(120.0, 365)
(4900.0, 28692)
(2240.0, 23228)
(190.0, 1683)
(540.0, 887)
(120.0, 401)
(900.0, 3093)
(600.0, 924)
(300.0, 2271)
(300.0, 812)
(200.0, 256)
(390.0, 408)
(2000.0, 15588)
(200.0, 1223)
(560.0, 1559)
(600.0, 2770)
(3700.0, 7939)
(200.0, 2124)
(900.0, 3146)
(3600.0, 14451)
(80.0, 249)
(155.0, 861)
(240.0, 656)
(560.0, 4104)
(3400.0, 31555)
(600.0, 2997)
(2400.0, 17720)
(560.0, 3844)
(1800.0, 11336)
(3720.0, 38732)
(2000.0, 19223)
(150.0, 1379)
(190.0, 506)
(285.0, 1255)
(6470.0, 64438)
(60.0, 419)
(60.0, 639)
(1400.0, 6453)
(115.0, 543)
(600.0, 5501)
(3700.0, 24435)
(3700.0, 25415)
(2260.0, 14332)
(6470.0, 66148)
(1640.0, 13610)
(1800.0, 3